# ⛓ Chain 

In [ ]:
#| default_exp chains

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import asyncio, os
from functools import wraps, lru_cache
from async_lru import alru_cache
from cachetools import cached, TTLCache
from typing import List, TypeVar, Callable, Optional, Tuple, Dict
from web3 import Web3, HTTPProvider, AsyncWeb3, AsyncHTTPProvider, Account
from web3.eth.async_eth import AsyncContract
from web3.eth import Contract
from web3.manager import RequestManager, RequestBatcher
from sugar.config import ChainSettings, make_op_chain_settings, make_base_chain_settings
from sugar.helpers import normalize_address, MAX_UINT256, float_to_uint256, apply_slippage, get_future_timestamp
from sugar.abi import get_abi
from sugar.token import Token
from sugar.pool import LiquidityPool, LiquidityPoolForSwap
from sugar.price import Price
from sugar.deposit import Deposit
from sugar.helpers import ADDRESS_ZERO, chunk, normalize_address, Pair, find_all_paths
from sugar.quote import QuoteInput, Quote
from sugar.swap import setup_planner

In [ ]:
#| export

# monkey patching how web3 handles errors in batched requests
# re: https://github.com/ethereum/web3.py/issues/3657
original_format_batched_response = RequestManager._format_batched_response
def safe_format_batched_response(*args):
    try: return original_format_batched_response(*args)
    except Exception as e: return e
RequestManager._format_batched_response = safe_format_batched_response

## Chain implementation 

In [ ]:
#| export

T = TypeVar('T')

def require_context(f: Callable[..., T]) -> Callable[..., T]:
    @wraps(f)
    def wrapper(self: 'CommonChain', *args, **kwargs) -> T:
        if not self._in_context: raise RuntimeError("Chain methods can only be accessed within 'async with' block")
        return f(self, *args, **kwargs)
    return wrapper

def require_async_context(f: Callable[..., T]) -> Callable[..., T]:
    @wraps(f)
    async def wrapper(self: 'CommonChain', *args, **kwargs) -> T:
        if not self._in_context: raise RuntimeError("Chain methods can only be accessed within 'async with' block")
        return await f(self, *args, **kwargs)
    return wrapper


class CommonChain:
    @property
    def account(self) -> Account: return self.web3.eth.account.from_key(os.getenv("SUGAR_PK"))

    pools: Optional[List[LiquidityPool]] = None
    pools_for_swap: Optional[List[LiquidityPoolForSwap]] = None

    def __init__(self, settings: ChainSettings, **kwargs):
        self.settings, self._in_context = settings, False

        if "pools" in kwargs: self.pools = kwargs["pools"]
        if "pools_for_swap" in kwargs: self.pools_for_swap = kwargs["pools_for_swap"] 
    
    def prepare_set_token_allowance_contract(self, token: Token, contract_wrapper):
        ERC20_ABI = [{
            "name": "approve",
            "type": "function",
            "constant": False,
            "inputs": [{"name": "spender", "type": "address"}, {"name": "amount", "type": "uint256"}],
            "outputs": [{"name": "", "type": "bool"}]
        }]
        return contract_wrapper(address=token.wrapped_token_address or token.token_address, abi=ERC20_ABI)
    
    def prepare_tokens(self, tokens: List[Tuple], listed_only: bool) -> List[Token]:
        native = Token.make_native_token(self.settings.native_token_symbol, self.settings.wrapped_native_token_addr, self.settings.native_token_decimals)
        ts = list(map(lambda t: Token.from_tuple(t), tokens))
        return [native] + (list(filter(lambda t: t.listed, ts)) if listed_only else ts)
    
    def _prepare_prices(self, tokens: List[Token], rates: List[int]) -> Dict[str, int]:
        # token_address => normalized rate
        result = {}
        # rates are returned multiplied by eth decimals + the difference in decimals to eth
        # we want them all normalized to 18 decimals
        for cnt, rate in enumerate(rates):
            t, eth_decimals = tokens[cnt], self.settings.native_token_decimals
            if t.decimals == eth_decimals: nr = rate
            elif t.decimals < eth_decimals: nr = rate // (10 ** (eth_decimals - t.decimals))
            else: nr = rate * (10 ** (t.decimals - eth_decimals))
            result[t.token_address] = nr
        return result

    def prepare_prices(self, tokens: List[Token], prices: List[int]) -> List[Price]:
        """Get prices for tokens in target stable token"""
        eth_decimals = self.settings.native_token_decimals
        # all rates in EHT: token => rate
        rates_in_eth = self._prepare_prices(tokens, prices)
        eth_rate, usd_rate = rates_in_eth[self.settings.native_token_symbol], rates_in_eth[self.settings.stable_token_addr]
        # this gives us the price of 1 eth in usd with 18 decimals precision
        eth_usd_price = (eth_rate * 10 ** eth_decimals) // usd_rate
        # finally convert to prices in terms of stable
        return [Price(token=t, price=(rates_in_eth[t.token_address] * eth_usd_price // 10 ** eth_decimals) / 10 ** eth_decimals) for t in tokens]
    
    def prepare_pools(self, pools: List[Tuple], tokens: List[Token], prices: List[Price]) -> List[LiquidityPool]:
        tokens, prices = {t.token_address: t for t in tokens}, {price.token.token_address: price for price in prices}
        return list(filter(lambda p: p is not None, map(lambda p: LiquidityPool.from_tuple(p, tokens, prices), pools)))
    
    def prepare_pools_for_swap(self, pools: List[Tuple]) -> List[LiquidityPoolForSwap]:
        return list(map(lambda p: LiquidityPoolForSwap.from_tuple(p), pools))
    
    def filter_pools_for_swap(self, pools: List[LiquidityPoolForSwap], from_token: Token, to_token: Token) -> List[LiquidityPoolForSwap]:
        match_tokens = set(self.settings.connector_tokens_addrs + [from_token.token_address, to_token.token_address])
        return list(filter(lambda p: p.token0_address in match_tokens or p.token1_address in match_tokens, pools))
    
    def paths_to_pools(self, pools: List[LiquidityPoolForSwap], paths: List[List[Tuple]]) -> List[LiquidityPoolForSwap]:
        pools_dict = {p.lp: p for p in pools}
        return [list(map(lambda p: pools_dict[p[2]], path)) for path in paths]

    def prepare_quote_batch(self, from_token: Token, to_token: Token, batcher: RequestBatcher, pools: List[List[LiquidityPoolForSwap]], amount_in: int, paths: List[List[Tuple]]):
        inputs = []
        for i, path in enumerate(paths):
            p = [(p, p.token0_address != path[i][0]) for i, p in enumerate(pools[i])]
            q = QuoteInput(from_token=from_token, to_token=to_token, amount_in=amount_in, path=p)
            batcher.add(self.quoter.functions.quoteExactInput(q.route.encoded, amount_in))
            inputs.append(q)
        return batcher, inputs

    def prepare_quotes(self, quote_inputs: List[QuoteInput], responses):
        if len(responses) != len(quote_inputs): raise ValueError(f"Number of responses {len(responses)} does not match number of quote inputs {len(quote_inputs)}")
        quotes = []
        for i, r in enumerate(responses):
            if isinstance(r, Exception): continue
            else: quotes.append(Quote(input=quote_inputs[i], amount_out=r[0]))
        return quotes
    
    def get_paths_for_quote(self, from_token: Token, to_token: Token, pools: List[LiquidityPoolForSwap], exclude_tokens: List[str]) -> List[List[Tuple]]:
        exclude_tokens_set = set(map(lambda t: normalize_address(t), exclude_tokens))

        if from_token.token_address in exclude_tokens: exclude_tokens_set.remove(from_token.token_address)
        if to_token.token_address in exclude_tokens: exclude_tokens_set.remove(to_token.token_address)

        pairs = [Pair(p.token0_address, p.token1_address, p.lp) for p in pools]
        paths = find_all_paths(pairs, from_token.wrapped_token_address or from_token.token_address, to_token.wrapped_token_address or to_token.token_address)
        # filter out paths with excluded tokens
        return list(filter(lambda p: len(set(map(lambda t: t[0], p)) & exclude_tokens_set) == 0, paths))

### Aync chain

In [ ]:
#| export

class AsyncChain(CommonChain):
    web3: AsyncWeb3
    sugar: AsyncContract
    slipstream: AsyncContract
    router: AsyncContract
    prices: AsyncContract
    quoter: AsyncContract
    swapper: AsyncContract


    async def __aenter__(self):
        """Async context manager entry"""
        self._in_context = True
        self.web3 = AsyncWeb3(AsyncHTTPProvider(self.settings.rpc_uri))
        self.sugar = self.web3.eth.contract(address=self.settings.sugar_contract_addr, abi=get_abi("sugar"))
        self.slipstream = self.web3.eth.contract(address=self.settings.slipstream_contract_addr, abi=get_abi("slipstream"))
        self.prices = self.web3.eth.contract(address=self.settings.price_oracle_contract_addr, abi=get_abi("price_oracle"))
        self.router = self.web3.eth.contract(address=self.settings.router_contract_addr, abi=get_abi("router"))
        self.quoter = self.web3.eth.contract(address=self.settings.quoter_contract_addr, abi=get_abi("quoter"))
        self.swapper = self.web3.eth.contract(address=self.settings.swapper_contract_addr, abi=get_abi("swapper"))

        # set up caching for price oracle
        self._get_prices = alru_cache(ttl=self.settings.pricing_cache_timeout_seconds)(self._get_prices)

        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """Async context manager exit"""
        self._in_context = False
        await self.web3.provider.disconnect()
        return None

    @require_async_context
    @alru_cache(maxsize=None)
    async def get_all_tokens(self, listed_only: bool = True) -> List[Token]:
        # TODO: pagination for tokens
        tokens = await self.sugar.functions.tokens(1000, 0, ADDRESS_ZERO, []).call()
        return self.prepare_tokens(tokens, listed_only)
    
    async def _get_prices(self, tokens: Tuple[Token]) -> List[int]:
        # token_address => normalized rate
        return await self.prices.functions.getManyRatesToEthWithCustomConnectors(
            list(map(lambda t: t.wrapped_token_address or t.token_address, tokens)),
            False, # use wrappers
            self.settings.connector_tokens_addrs,
            10 # threshold_filter
        ).call()

    @require_async_context
    async def get_prices(self, tokens: List[Token]) -> List[Price]:
        """Get prices for tokens in target stable token"""

        # batches starts as a list of lists 
        batches = await asyncio.gather(
            *map(
                # XX: lists are not cacheable, convert them to tuples so lru cache is happy
                lambda ts: self._get_prices(tuple(ts)),
                list(chunk(tokens, self.settings.price_batch_size)),
            )
        )
        return self.prepare_prices(tokens, sum(batches, []))

    @alru_cache(maxsize=None)
    async def get_raw_pools(self, for_swaps: bool):
        pools, offset, limit = [], 0, self.settings.pool_page_size
        while True:
            f = self.sugar.functions.all if not for_swaps else self.sugar.functions.forSwaps
            pools_batch = await f(limit, offset).call()
            pools += pools_batch
            if len(pools_batch) == 0: break
            else: offset += limit
        return pools
    
    @require_async_context
    async def get_pools(self, for_swaps: bool = False) -> List[LiquidityPool]:
        pools = await self.get_raw_pools(for_swaps)
        if not for_swaps:
            tokens = await self.get_all_tokens()
            return self.prepare_pools(pools, tokens, await self.get_prices(tokens))
        else: return self.prepare_pools_for_swap(pools)
    
    @require_async_context
    @alru_cache(maxsize=None)
    async def get_pool_by_address(self, address: str) -> Optional[LiquidityPool]:
        try:
            p = await self.sugar.functions.byAddress(address).call()
        except: return None
        tokens = await self.get_all_tokens(listed_only=False)
        return self.prepare_pools([p], tokens, await self.get_prices(tokens))[0]
    
    @require_async_context
    async def get_pools_for_swaps(self) -> List[LiquidityPoolForSwap]: return await self.get_pools(for_swaps=True)

    @require_async_context
    async def _get_quotes_for_paths(self, from_token: Token, to_token: Token, amount_in: int, pools: List[LiquidityPoolForSwap], paths: List[List[Tuple]]) -> List[Optional[Quote]]:
        path_pools = self.paths_to_pools(pools, paths)
        async with self.web3.batch_requests() as batch:
            batch, inputs = self.prepare_quote_batch(from_token, to_token, batch, path_pools, amount_in, paths)
            return self.prepare_quotes(inputs, await batch.async_execute())

    @require_async_context
    async def get_quote(self, from_token: Token, to_token: Token, amount: float, filter_quotes: Optional[Callable[[Quote], bool]] = None) -> Optional[Quote]:
        amount_in = amount
        pools = self.filter_pools_for_swap(from_token=from_token, to_token=to_token, pools=await self.get_pools_for_swaps())
        paths = self.get_paths_for_quote(from_token, to_token, pools, self.settings.excluded_tokens_addrs)
        # TODO: investigate why this takes too long
        # quotes = sum(await asyncio.gather(*[self._get_quotes_for_paths(from_token, to_token, amount_in, pools, paths) for paths in chunk(paths, 500)]), [])
        quotes = [] 
        for paths in chunk(paths, 500):
            r = await self._get_quotes_for_paths(from_token, to_token, float_to_uint256(amount_in, decimals=from_token.decimals), pools, paths)
            for q in r:
                if q is not None: quotes.append(q)
        if filter_quotes is not None: quotes = list(filter(filter_quotes, quotes))
        return max(quotes, key=lambda q: q.amount_out) if len(quotes) > 0 else None
    
    @require_async_context
    async def swap(self, from_token: Token, to_token: Token, amount: float, slippage: Optional[float] = None):
        q = await self.get_quote(from_token, to_token, amount)
        if not q: raise ValueError("No quotes found")
        return await self.swap_from_quote(q, slippage=slippage)
        
    @require_async_context
    async def swap_from_quote(self, quote: Quote, slippage: Optional[float] = None):
        swapper_contract_addr, from_token = self.settings.swapper_contract_addr, quote.from_token
        planner = setup_planner(quote=quote, slippage=slippage if slippage is not None else self.settings.swap_slippage, account=self.account.address, router_address=swapper_contract_addr)
        await self.set_token_allowance(from_token, swapper_contract_addr, quote.input.amount_in)
        value = quote.input.amount_in if from_token.wrapped_token_address else 0
        return await self.sign_and_send_tx(self.swapper.functions.execute(*[planner.commands, planner.inputs]), value=value)

    @require_async_context
    async def set_token_allowance(self, token: Token, addr: str, amount: int):
        token_contract = self.prepare_set_token_allowance_contract(token, self.web3.eth.contract)
        return await self.sign_and_send_tx(token_contract.functions.approve(addr, amount))
        

    @require_async_context
    async def check_token_allowance(self, token: Token, addr: str) -> int:
        ERC20_ABI = [{
            "name": "allowance",
            "type": "function",
            "constant": True,
            "inputs": [{"name": "owner", "type": "address"}, {"name": "spender", "type": "address"}],
            "outputs": [{"name": "", "type": "uint256"}]
        }]
        token_contract = self.web3.eth.contract(address=token.wrapped_token_address or token.token_address, abi=ERC20_ABI)
        return await token_contract.functions.allowance(self.account.address, addr).call()

    @require_async_context
    async def sign_and_send_tx(self, tx, value: int = 0, wait: bool = True):
        spender = self.account.address
        tx = await tx.build_transaction({ 'from': spender, 'value': value, 'nonce': await self.web3.eth.get_transaction_count(spender) })
        signed_tx = self.account.sign_transaction(tx)
        tx_hash = await self.web3.eth.send_raw_transaction(signed_tx.raw_transaction)
        return await self.web3.eth.wait_for_transaction_receipt(tx_hash) if wait else tx_hash
    
    @require_async_context
    async def deposit(self, deposit: Deposit, delay_in_minutes: float = 30, slippage: float = 0.01):
        amount_token0, pool, router_contract_addr = deposit.amount_token0, deposit.pool, self.settings.router_contract_addr
        print(f"gonna deposit {amount_token0} {pool.token0.symbol} into {pool.symbol} from {self.account.address}")
        [token0_amount, token1_amount, _] = await self.router.functions.quoteAddLiquidity(
            pool.token0.token_address,
            pool.token1.token_address,
            pool.is_stable,
            pool.factory,
            float_to_uint256(amount_token0, pool.token0.decimals),
            MAX_UINT256
        ).call()
        print(f"Quote: {pool.token0.symbol} {token0_amount / 10 ** pool.token0.decimals} -> {pool.token1.symbol} {token1_amount / 10 ** pool.token1.decimals}")

        # set up allowance for both tokens
        print(f"setting up allowance for {pool.token0.symbol}")
        await self.set_token_allowance(pool.token0, router_contract_addr, token0_amount)

        print(f"setting up allowance for {pool.token1.symbol}")
        await self.set_token_allowance(pool.token1, router_contract_addr, token1_amount)

        # check allowances
        token0_allowance = await self.check_token_allowance(pool.token0, router_contract_addr)
        token1_allowance = await self.check_token_allowance(pool.token1, router_contract_addr)

        print(f"allowances: {token0_allowance}, {token1_allowance}")

        # adding liquidity

        # if token 0 is native, use addLiquidityETH instead of standard addLiquidity
        if pool.token0.token_address == self.settings.wrapped_native_token_addr:
            params = [
                pool.token1.token_address,
                pool.is_stable,
                token1_amount,
                apply_slippage(token1_amount, slippage),
                apply_slippage(token0_amount, slippage),
                self.account.address,
                get_future_timestamp(delay_in_minutes)
            ]
            print(f"adding liquidity with params: {params}")
            return await self.sign_and_send_tx(self.router.functions.addLiquidityETH(*params), value=token0_amount)
        
        # token 1 is native, use addLiquidityETH instead of standard addLiquidity
        if pool.token1.token_address == self.settings.wrapped_native_token_addr:
            params = [
                pool.token0.token_address,
                pool.is_stable,
                token0_amount,
                apply_slippage(token0_amount, slippage),
                apply_slippage(token1_amount, slippage),
                self.account.address,
                get_future_timestamp(delay_in_minutes)
            ]
            print(f"adding liquidity with params: {params}")
            return await self.sign_and_send_tx(self.router.functions.addLiquidityETH(*params), value=token1_amount)

        params = [
            pool.token0.token_address,
            pool.token1.token_address,
            pool.is_stable,
            token0_amount,
            token1_amount,
            apply_slippage(token0_amount, slippage),
            apply_slippage(token1_amount, slippage),
            self.account.address,
            get_future_timestamp(delay_in_minutes)
        ]

        print(f"adding liquidity with params: {params}")

        return await self.sign_and_send_tx(self.router.functions.addLiquidity(*params))

## Sync chain

In [ ]:
#| export

class Chain(CommonChain):
    web3: Web3
    sugar: Contract
    router: Contract
    slipstream: Contract
    prices: Contract
    quoter: Contract
    swapper: Contract

    def __enter__(self):
        """Sync context manager entry"""
        self._in_context = True
        self.web3 = Web3(HTTPProvider(self.settings.rpc_uri))
        self.sugar = self.web3.eth.contract(address=self.settings.sugar_contract_addr, abi=get_abi("sugar"))
        self.slipstream = self.web3.eth.contract(address=self.settings.slipstream_contract_addr, abi=get_abi("slipstream"))
        self.prices = self.web3.eth.contract(address=self.settings.price_oracle_contract_addr, abi=get_abi("price_oracle"))
        self.router = self.web3.eth.contract(address=self.settings.router_contract_addr, abi=get_abi("router"))
        self.quoter = self.web3.eth.contract(address=self.settings.quoter_contract_addr, abi=get_abi("quoter"))
        self.swapper = self.web3.eth.contract(address=self.settings.swapper_contract_addr, abi=get_abi("swapper"))

        # set up caching for price oracle
        self._get_prices = cached(TTLCache(ttl=self.settings.pricing_cache_timeout_seconds, maxsize=self.settings.price_batch_size * 10))(self._get_prices)

        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Sync context manager exit"""
        self._in_context = False
        return None
    
    @require_context
    def sign_and_send_tx(self, tx, value: int = 0, wait: bool = True):
        spender = self.account.address
        tx = tx.build_transaction({ 'from': spender, 'value': value, 'nonce': self.web3.eth.get_transaction_count(spender) })
        signed_tx = self.account.sign_transaction(tx)
        tx_hash = self.web3.eth.send_raw_transaction(signed_tx.raw_transaction)
        return self.web3.eth.wait_for_transaction_receipt(tx_hash) if wait else tx_hash
    
    @require_context
    def set_token_allowance(self, token: Token, addr: str, amount: int):
        token_contract = self.prepare_set_token_allowance_contract(token, self.web3.eth.contract)
        return self.sign_and_send_tx(token_contract.functions.approve(addr, amount))

    @require_context
    @lru_cache(maxsize=None)
    def get_all_tokens(self, listed_only: bool = True) -> List[Token]:
        return self.prepare_tokens(self.sugar.functions.tokens(800, 0, ADDRESS_ZERO, []).call(), listed_only)
    
    def _get_prices(self, tokens: Tuple[Token]) -> List[int]:
        # token_address => normalized rate
        return self.prices.functions.getManyRatesToEthWithCustomConnectors(
            list(map(lambda t: t.wrapped_token_address or t.token_address, tokens)),
            False, # use wrappers
            self.settings.connector_tokens_addrs,
            10 # threshold_filter
        ).call()

    @require_context
    def get_prices(self, tokens: List[Token]) -> List[Price]:
        """Get prices for tokens in target stable token"""
        return self.prepare_prices(tokens, sum([self._get_prices(tuple(ts)) for ts in list(chunk(tokens, self.settings.price_batch_size))], []))
    
    @lru_cache(maxsize=None)
    def get_raw_pools(self, for_swaps: bool):
        pools, offset, limit = [], 0, self.settings.pool_page_size
        while True:
            f = self.sugar.functions.all if not for_swaps else self.sugar.functions.forSwaps
            pools_batch = f(limit, offset).call()
            pools += pools_batch
            if len(pools_batch) == 0: break
            else: offset += limit
        return pools

    @require_context
    def get_pools(self, for_swaps: bool = False) -> List[LiquidityPool]:
        pools = self.get_raw_pools(for_swaps)
        if not for_swaps:
            tokens = self.get_all_tokens(listed_only=False)
            return self.prepare_pools(pools, tokens, self.get_prices(tokens))
        else: return self.prepare_pools_for_swap(pools)

    @require_context
    @lru_cache(maxsize=None)
    def get_pool_by_address(self, address: str) -> Optional[LiquidityPool]:
        try:
            p = self.sugar.functions.byAddress(address).call()
        except: return None
        tokens = self.get_all_tokens(listed_only=False)
        return self.prepare_pools([p], tokens, self.get_prices(tokens))[0]
    
    @require_context
    def get_pools_for_swaps(self) -> List[LiquidityPoolForSwap]: return self.get_pools(for_swaps=True)

    @require_context
    def _get_quotes_for_paths(self, from_token: Token, to_token: Token, amount_in: int, pools: List[LiquidityPoolForSwap], paths: List[List[Tuple]]) -> List[Optional[Quote]]:
        path_pools = self.paths_to_pools(pools, paths)
        with self.web3.batch_requests() as batch:
            batch, inputs = self.prepare_quote_batch(from_token, to_token, batch, path_pools, amount_in, paths)
            return self.prepare_quotes(inputs, batch.execute())

    @require_context
    def get_quote(self, from_token: Token, to_token: Token, amount: float, filter_quotes: Optional[Callable[[Quote], bool]] = None) -> Optional[Quote]:
        amount_in = amount
        pools = self.filter_pools_for_swap(from_token=from_token, to_token=to_token, pools=self.get_pools_for_swaps())
        paths = self.get_paths_for_quote(from_token, to_token, pools, self.settings.excluded_tokens_addrs)
        path_chunks, chain_instance = list(chunk(paths, 500)), self
        def get_quotes_for_chunk(paths_chunk): return chain_instance._get_quotes_for_paths(from_token, to_token, float_to_uint256(amount_in, decimals=from_token.decimals), pools, paths_chunk)
        # TODO: make this work in threads re: https://github.com/ethereum/web3.py/issues/3613
        all_quotes = []
        for pc in path_chunks:
            quotes = get_quotes_for_chunk(pc)
            for q in quotes: all_quotes.append(q)

        if filter_quotes is not None: all_quotes = list(filter(filter_quotes, all_quotes))

        return max(all_quotes, key=lambda q: q.amount_out) if len(all_quotes) > 0 else None
    
    @require_context
    def swap(self, from_token: Token, to_token: Token, amount: float, slippage: Optional[float] = None):
        q = self.get_quote(from_token, to_token, amount)
        if not q: raise ValueError("No quotes found")
        return self.swap_from_quote(q, slippage=slippage)
        
    @require_context
    def swap_from_quote(self, quote: Quote, slippage: Optional[float] = None):
        swapper_contract_addr, from_token = self.settings.swapper_contract_addr, quote.from_token
        planner = setup_planner(quote=quote, slippage=slippage if slippage is not None else self.settings.swap_slippage, account=self.account.address, router_address=swapper_contract_addr)
        self.set_token_allowance(from_token, swapper_contract_addr, quote.input.amount_in)
        value = quote.input.amount_in if from_token.wrapped_token_address else 0
        return self.sign_and_send_tx(self.swapper.functions.execute(*[planner.commands, planner.inputs]), value=value)

## OP Chain

In [ ]:
#| export

class OPChainCommon():
    usdc: Token = Token(token_address='0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85', symbol='USDC', decimals=6, listed=True, wrapped_token_address=None)
    velo: Token = Token(token_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', symbol='VELO', decimals=18, listed=True, wrapped_token_address=None)
    eth: Token = Token(token_address='ETH', symbol='ETH', decimals=18, listed=True, wrapped_token_address='0x4200000000000000000000000000000000000006')
    
class AsyncOPChain(AsyncChain, OPChainCommon):
    def __init__(self, **kwargs): super().__init__(make_op_chain_settings(**kwargs), **kwargs)

class OPChain(Chain, OPChainCommon):
    def __init__(self, **kwargs): super().__init__(make_op_chain_settings(**kwargs), **kwargs)


## Base Chain

In [ ]:
#| export

class BaseChainCommon():
    usdc: Token = Token(token_address='0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913', symbol='USDC', decimals=6, listed=True, wrapped_token_address=None)
    aero: Token = Token(token_address='0x940181a94A35A4569E4529A3CDfB74e38FD98631', symbol='AERO', decimals=18, listed=True, wrapped_token_address=None)
    eth: Token = Token(token_address='ETH', symbol='ETH', decimals=18, listed=True, wrapped_token_address='0x4200000000000000000000000000000000000006')
    
class AsyncBaseChain(AsyncChain, BaseChainCommon):
    def __init__(self, **kwargs): super().__init__(make_base_chain_settings(**kwargs), **kwargs)

class BaseChain(Chain, BaseChainCommon):
    def __init__(self, **kwargs): super().__init__(make_base_chain_settings(**kwargs), **kwargs)

## Simnet versions of chains

Simnet URIs:

- OP: http://127.0.0.1:4444
- Base: http://127.0.0.1:4445

This assumes the following setup:

```
supersim fork  --l2.host=0.0.0.0 --l2.starting.port=4444 --chains=op,base
```

For local dev, use:

```
python ./supersim.py
```

In [ ]:
class AsyncOPChainSimnet(AsyncOPChain):
    def __init__(self,  **kwargs): super().__init__(rpc_uri="http://127.0.0.1:4444", **kwargs)

class AsyncBaseChainSimnet(AsyncBaseChain):
    def __init__(self,  **kwargs): super().__init__(rpc_uri="http://127.0.0.1:4445", **kwargs)

class OPChainSimnet(OPChain):
    def __init__(self,  **kwargs): super().__init__(rpc_uri="http://127.0.0.1:4444", **kwargs)

class BaseChainSimnet(BaseChain):
    def __init__(self,  **kwargs): super().__init__(rpc_uri="http://127.0.0.1:4445", **kwargs)

## Tests

Run tests using mainnets for reads and simnet for writes

Make sure simnet is running

In [ ]:
import socket
from fastcore.test import test_close
from fastcore.test import test_eq

In [ ]:
host, port ='127.0.0.1', 4444
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex((host, port))
    # are you running supersim?
    test_eq(result, 0)
except socket.error as err:
    test_eq(err, None)
finally:
    sock.close()

Make sure PK is set for writes

In [ ]:
import os
from fastcore.test import test_ne
from dotenv import load_dotenv

load_dotenv()

test_ne(os.getenv("SUGAR_PK"), None)

Check simnet URLs

In [ ]:
async with AsyncOPChainSimnet() as op:
    assert "127.0.0.1" in op.settings.rpc_uri
async with AsyncBaseChainSimnet() as base:
    assert "127.0.0.1" in base.settings.rpc_uri
with OPChainSimnet() as op:
    assert "127.0.0.1" in op.settings.rpc_uri
with BaseChainSimnet() as base:
    assert "127.0.0.1" in base.settings.rpc_uri

Check real chain URLs - let's make sure we are not running on public stuff

In [ ]:
async with AsyncOPChain() as op:
    assert op.settings.rpc_uri != "https://mainnet.base.org"

async with AsyncBaseChain() as base:
    assert base.settings.rpc_uri != "https://optimism-mainnet.wallet.coinbase.com"

Getting tokens. Make sure native token is included

In [ ]:
from fastcore.test import test_eq, test_ne

async with AsyncOPChain() as op:
    tokens = await op.get_all_tokens()
    native_token = next(filter(lambda t: t.is_native, tokens), None)
    test_ne(native_token, None)
    test_eq(native_token.symbol, "ETH")

with OPChain() as op_sync:
    tokens = op_sync.get_all_tokens()
    native_token = next(filter(lambda t: t.is_native, tokens), None)
    test_ne(native_token, None)
    test_eq(native_token.symbol, "ETH")

async with AsyncBaseChain() as base:
    tokens = await base.get_all_tokens()
    native_token = next(filter(lambda t: t.is_native, tokens), None)
    test_ne(native_token, None)
    test_eq(native_token.symbol, "ETH")

Check on tokens and pools

In [ ]:
async with AsyncOPChain() as chain:
    tokens = await chain.get_all_tokens()
    prices = await chain.get_prices(tokens)
    pools = await chain.get_pools()    

    for p in prices[:5]: print(f"{p.token.symbol} price: ${p.price}")
    for p in pools[:5]: print(f"{p.symbol} pool: ${p.tvl}")

    # getting pool by ID
    first_pool = pools[0]
    p = await chain.get_pool_by_address(first_pool.lp)

    test_eq(first_pool.lp, p.lp)
    test_eq(first_pool.symbol, p.symbol)

    # not a pool
    p = await chain.get_pool_by_address(first_pool.factory)
    test_eq(p, None)    

ETH price: $1961.3608156633686
VELO price: $0.0523488899415954
RED price: $0.08541200422841895
USDC price: $1.0
WETH price: $1961.3608156633686
vAMM-RED/VELO pool: $157750.19261723245
vAMM-USDC/VELO pool: $326751.57601649285
vAMM-WETH/VELO pool: $333628.926451394
sAMM-alETH/frxETH pool: $556936.6034547389
vAMM-wstETH/LDO pool: $96148.94900993176


In [ ]:
with OPChain() as chain:
    tokens = chain.get_all_tokens()
    prices = chain.get_prices(tokens)
    pools = chain.get_pools()

    for p in prices[:5]: print(f"{p.token.symbol} price: ${p.price}")
    for p in pools[:5]: print(f"{p.symbol} pool: ${p.tvl}")

    # getting pool by ID
    first_pool = pools[0]
    p = chain.get_pool_by_address(first_pool.lp)

    test_eq(first_pool.lp, p.lp)
    test_eq(first_pool.symbol, p.symbol)

    # not a pool
    p = chain.get_pool_by_address(first_pool.factory)
    test_eq(p, None)

ETH price: $1961.3608156633686
VELO price: $0.0523488899415954
RED price: $0.08541200422841895
USDC price: $1.0
WETH price: $1961.3608156633686
sAMM-opxveVELO/VELO pool: $215.9457069895265
vAMM-RED/VELO pool: $157750.19261723245
vAMM-USDC/VELO pool: $326751.57601649285
vAMM-UNLOCK/VELO pool: $314.2069707093026
vAMM-WETH/VELO pool: $333628.926451394


### Swaps

Let's test some swaps here

In [ ]:
# velo -> usdc
# async with AsyncOPChain() as op:
#     quote = await op.get_quote(from_token=op.velo, to_token=op.usdc, amount=10)
    
#     async with AsyncOPChainSimnet() as op_simnet:
#         print(await op_simnet.swap_from_quote(quote))

#     with OPChainSimnet() as op_simnet_sync:
#         print(op_simnet_sync.swap_from_quote(quote))

# # velo -> eth
# async with AsyncOPChain() as op:
#     quote = await op.get_quote(op.velo, op.eth, 10)
#     async with AsyncOPChainSimnet() as op_simnet:
#         print(await op_simnet.swap_from_quote(quote))

# # eth -> velo
# async with AsyncOPChain() as op:
#     quote = await op.get_quote(op.eth, op.velo, 0.001)
#     async with AsyncOPChainSimnet() as op_simnet:
#         print(await op_simnet.swap_from_quote(quote))


AttributeDict({'type': 2, 'status': 1, 'cumulativeGasUsed': 464957, 'logs': [AttributeDict({'address': '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 'topics': [HexBytes('0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'), HexBytes('0x0000000000000000000000001e7a6b63f98484514610a9f0d5b399d4f7a9b1da'), HexBytes('0x0000000000000000000000004bf3e32de155359d1d75e8b474b66848221142fc')], 'data': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), 'blockHash': HexBytes('0x9828bc89b0cd1d67bed2671ec6ed1fc852be6bedd3994f152a8e290ac3362dd2'), 'blockNumber': 135557922, 'blockTimestamp': '0x681cc030', 'transactionHash': HexBytes('0xade95a886730e9648f37a79f536f0714c40107d237daa03b6665e70dba1c62b3'), 'transactionIndex': 0, 'logIndex': 0, 'removed': False}), AttributeDict({'address': '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('0x0000000000000000000000001e7a

Let's make sure sync VS async quote return consistent results

In [ ]:
# velo -> usdc

async with AsyncOPChain() as op:
    q1 = await op.get_quote(from_token=op.velo, to_token=op.usdc, amount=10)
    amount_out1 = q1.amount_out / 10 ** op.usdc.decimals


with OPChain() as op:
    q2 = op.get_quote(from_token=op.velo, to_token=op.usdc, amount=10)
    amount_out2 = q2.amount_out / 10 ** op.usdc.decimals

test_close(amount_out1, amount_out2, 0.01)

Basic deposit on OP

In [ ]:
# async with AsyncOPChain() as chain:
#     pools = await chain.get_pools()
#     pools = list(filter(lambda x: x.token0 and x.token0.token_address == chain.usdc and x.token1.token_address == chain.velo, pools))
#     async with AsyncOPChainSimnet() as supersim:
#         print(f"supersim is {supersim}")
#         # 0.02 USDC 
#         await supersim.deposit(Deposit(pools[0], 0.02))

Native -> non Native on OP

In [ ]:
## WIP
# async with OPChain() as op:
#     pools = await op.get_pools()
#     # find WETH/Velo pool
#     weth_velo_pool = next(filter(lambda p: p.token0.token_address == op.settings.wrapped_native_token_addr and p.token1.token_address == OPChain.velo, pools), None)
#     test_ne(weth_velo_pool, None)
#     async with OPChainSimnet() as supersim:
#         await supersim.deposit(Deposit(weth_velo_pool, 0.0001))

Basic deposit on Base

In [ ]:
# async with AsyncBaseChain() as chain:
#     pools = await chain.get_pools()
#     pools = list(filter(lambda x: x.token0 and x.token0.token_address == chain.usdc and x.token1.token_address == chain.aero, pools))
#     async with AsyncBaseChainSimnet() as supersim:
#         # 0.02 USDC 
#         await supersim.deposit(Deposit(pools[0], 0.01), slippage=0.05)

CL on OP

In [ ]:
# WIP
# async with OPChain() as chain:
#     pools = await chain.get_pools()
#     # CL200-USDC/VELO
#     cl_usdc_velo = next(filter(lambda x: x.token0 and x.token0.token_address == chain.usdc and x.token1.token_address == chain.velo and x.is_cl, pools), None)
#     test_ne(cl_usdc_velo, None)
#     async with OPChainSimnet() as supersim:
#         await supersim.deposit(Deposit(cl_usdc_velo, 0.02))


In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()